# DLVC WS 2017

Tutorial 8: Cost Functions and their Properties
==

Load Packages
==

In [ ]:
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_ssim

Load Data:
===============

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 100

trainset = torchvision.datasets.MNIST(root='./MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.MNIST(root='./MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

Define the Autoencoder:
===============

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 256),
            nn.ReLU())
        self.decoder = nn.Sequential(
            nn.Linear(256, 400),
            nn.ReLU(),
            nn.Linear(400, 28*28),
            nn.ReLU())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


net1 = autoencoder()
net2 = autoencoder()

if use_gpu:
    net1 = net1.double().cuda()
    net2 = net2.double().cuda()
else:
    net1 = net1.double()
    net2 = net2.double()
        

Train Autoencoder:
===========

In [ ]:
def Train(model,optimizer,criterion,IP,LB):
    optimizer.zero_grad()
    OP = model(IP)
    loss = criterion(OP, LB)
    loss.backward()
    optimizer.step()

In [ ]:
iterations = 10

optimizer1 = optim.Adam(net1.parameters(), lr=1e-4)
optimizer2 = optim.Adam(net2.parameters(), lr=1e-4)

criterion1 = nn.MSELoss()
criterion2 = nn.L1Loss()

Plotssim1 = []
Plotssim2 = []

dataiter = iter(testloader)
images, labels = dataiter.next()
if use_gpu:
    testinputs = Variable(images[0].view(-1, 28*28).double()).cuda()
else:
    testinputs = Variable(images[0].view(-1, 28*28).double()) 

for epoch in range(iterations):  # loop over the dataset multiple times
    ssim1 = 0.0
    ssim2 = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        if use_gpu:
            inputs = Variable(inputs.view(-1, 28*28).double()).cuda()
        else:
            inputs = Variable(inputs.view(-1, 28*28).double())
        Train(net1,optimizer1,criterion1,inputs,inputs)
        Train(net2,optimizer2,criterion2,inputs,inputs)        
    outputs = net1(testinputs)
    ssim1 = pytorch_ssim.ssim(Variable(testinputs.data.view(1,1,28,28).double()), Variable(outputs.data.view(1,1,28,28).double()))
    outputs = net2(testinputs)
    ssim2 = pytorch_ssim.ssim(Variable(testinputs.data.view(1,1,28,28).double()), Variable(outputs.data.view(1,1,28,28).double()))
    Plotssim1.append(float(ssim1))
    Plotssim2.append(float(ssim2))
    print('At Epoch %d ; SSIM Index (MSE):  %f ; SSIM Index (L1):  %f '%((epoch+1),ssim1,ssim2))
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
fig = plt.figure()         
plt.plot(range(epoch+1),Plotssim1,'r-',label='SSIM Index (MSE)')
plt.plot(range(epoch+1),Plotssim2,'g-',label='SSIM Index (L1)')      
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Testing SSIM')  
print('Finished Training')

Neural Network: 
================================

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.Layer1 = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 256),
            nn.ReLU())
        self.Layer2 = nn.Sequential(
            nn.Linear(256, 10))

    def forward(self, x):
        x = self.Layer1(x)
        x = self.Layer2(x)
        return x


net1 = NeuralNet()
net2 = NeuralNet()
net3 = NeuralNet()

if use_gpu:
    net1 = net1.double().cuda()
    net2 = net2.double().cuda()
    net3 = net3.double().cuda()
else:
    net1 = net1.double()
    net2 = net2.double()
    net3 = net3.double()

Train Classifier:
===========

In [ ]:
def TrainClassifier(model,optimizer,criterion,IP,LB,lossType):
    optimizer.zero_grad()
    OP = model(IP)
    
    if lossType=='CEL':
        loss = criterion(OP, LB)        
    else:
        loss = criterion(F.log_softmax(OP), LB)        
    loss.backward()
    optimizer.step()

In [ ]:
iterations = 10
learning_rate = 0.9
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
criterion3 = nn.MultiMarginLoss()

optimizer1 = optim.SGD(net1.parameters(), lr=1e-3)
optimizer2 = optim.SGD(net2.parameters(), lr=1e-3)
optimizer3 = optim.SGD(net3.parameters(), lr=1e-3)

Plotacc1 = []
Plotacc2 = []
Plotacc3 = []

for epoch in range(iterations):  # loop over the dataset multiple times

    correct1 = 0
    correct2 = 0
    correct3 = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()).cuda(), Variable(labels).cuda()
        else:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()), Variable(labels)
            
        TrainClassifier(net1,optimizer1,criterion1,inputs,labels,'CEL')
        TrainClassifier(net2,optimizer2,criterion2,inputs,labels,'NLL')
        TrainClassifier(net3,optimizer3,criterion3,inputs,labels,'MML')
        
    for data in testloader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()).cuda(), labels.cuda()
        else:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()), labels
        total += labels.size(0)
        
        outputs = net1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct1 += (predicted == labels).sum()
        
        outputs = net2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct2 += (predicted == labels).sum()
        
        outputs = net3(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct3 += (predicted == labels).sum()

    Plotacc1.append(correct1/float(total))
    Plotacc2.append(correct2/float(total))
    Plotacc3.append(correct3/float(total))
    print('At Epoch %d ;Acc (CrossEntropy):  %f ;Acc (NegativeLogLikelihood):  %f ;Acc (MultiMarginLoss):  %f'%((epoch+1),correct1/float(total),correct2/float(total),correct3/float(total)))
fig = plt.figure()        
plt.plot(range(epoch+1),Plotacc1,'r-',label='Cross Entropy Loss')
plt.plot(range(epoch+1),Plotacc2,'g-',label='Negative Log Likelihood Loss')   
plt.plot(range(epoch+1),Plotacc3,'b-',label='Multi Margin Loss')  
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Testing Accuracy')  
print('Finished Training')